- first modelling, no FE
- consider targets with only one positive label

In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from tqdm._tqdm_notebook import tqdm_notebook

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  import sys


In [2]:
DIR = "/kaggle/input/lish-moa/"
train_feat = pd.read_csv(DIR+"train_features.csv")
test_feat = pd.read_csv(DIR+"test_features.csv")
#train_nonscore = pd.read_csv(DIR+"train_targets_nonscored.csv")
train_score = pd.read_csv(DIR+"train_targets_scored.csv")
sub = pd.read_csv(DIR+"sample_submission.csv")

In [3]:
target_feats = [ i for i in train_score.columns if i != "sig_id"]
g_feats = [i for i in train_feat.columns if "g-" in i]
c_feats = [i for i in train_feat.columns if "c-" in i]

In [4]:
noncons_train_index = train_feat[train_feat.cp_type=="ctl_vehicle"].index
noncons_test_index = test_feat[test_feat.cp_type=="ctl_vehicle"].index

In [5]:
def g_calc(row):
    return len(np.unique(row[g_feats].values))

def c_calc(row):
    return len(np.unique(row[c_feats].values))

tqdm_notebook.pandas(desc="progress")
train_feat["g_unique_num"] = train_feat.progress_apply(g_calc, axis=1)
test_feat["g_unique_num"] = test_feat.progress_apply(g_calc, axis=1)

#train_feat["c_unique_num"] = train_feat.progress_apply(c_calc, axis=1)
#test_feat["c_unique_num"] = test_feat.progress_apply(c_calc, axis=1)

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [6]:
categoricals = ["cp_type", "cp_time", "cp_dose"]
lgbm_params = {'objective': 'binary', 'metric': 'binary_logloss', 'eval_metric': 'logloss', 'boosting_type': 'gbdt', 'tree_learner': 'serial', 'learning_rate': 0.01, "num_leaves": 10, 'random_seed':44,'max_depth': 5}

def modelling_lgb(new_train, target_train, new_test, target):
    
    X_train = new_train.drop(['sig_id'],axis=1).copy()
    y_train = target_train[target].copy()
    X_test = new_test.copy()
    X_test = new_test.drop(['sig_id'],axis=1).copy()
    
    for feat in categoricals:
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[feat]))
        X_train[feat] = lbl.transform(list(X_train[feat]))
        X_test[feat] = lbl.transform(list(X_test[feat]))
        
    pred_value = np.zeros(X_test.shape[0])

    #remove_features = []
    #for i in X_train.columns:
    #    if (X_train[i].std() == 0) and i not in remove_features:
    #        remove_features.append(i)
    #X_train = X_train.drop(remove_features, axis=1)
    #X_test = X_test.drop(remove_features, axis=1)
    
    features_list = [i for i in X_train.columns]
    feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
    
    if target not in ["erbb2_inhibitor", "atp-sensitive_potassium_channel_antagonist"]:
        n_folds=5
        skf=StratifiedKFold(n_splits = n_folds, shuffle=True, random_state=0)
        models = []

        valid = np.zeros([X_train.shape[0]])
        mean_score = 0
        for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
            print("Fold "+str(i+1))
            X_train2 = X_train.iloc[train_index,:]
            y_train2 = y_train.iloc[train_index]

            X_valid2 = X_train.iloc[test_index,:]
            y_valid2 = y_train.iloc[test_index]
        
            lgb_train = lgb.Dataset(X_train2, y_train2)
            lgb_eval = lgb.Dataset(X_valid2, y_valid2, reference=lgb_train)
        
            clf = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
               num_boost_round=10000,early_stopping_rounds=50,verbose_eval = 1000) 

            valid_predict = clf.predict(X_valid2, num_iteration = clf.best_iteration)
            mean_score += log_loss(y_valid2,valid_predict) / n_folds
            valid[test_index] = valid_predict
            feature_importance_df["Fold_"+str(i+1)] = clf.feature_importance()
            models.append(clf)
        
        feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
        feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
        feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]
        feature_importance_df = feature_importance_df.sort_values("Average", ascending=False).reset_index(drop=True)

        score = log_loss(y_train, valid)
        print("mean log_loss: {}, oof log_loss= {} ".format(mean_score, score))
        for model in models:
            pred_value += model.predict(X_test, num_iteration = model.best_iteration) / len(models)
    
    else:
        lgb_train = lgb.Dataset(X_train, y_train)
        clf = lgb.train(lgbm_params, lgb_train, num_boost_round=1000) 
        valid = clf.predict(X_train)
        mean_score = log_loss(y_train, valid) 
        score = log_loss(y_train, valid) 
        feature_importance_df["Average"] = clf.feature_importance()
        feature_importance_df = feature_importance_df.sort_values("Average", ascending=False).reset_index(drop=True)
        pred_value = clf.predict(X_test)
    
    return valid, pred_value, feature_importance_df, mean_score, score

train_checkscore = train_score.copy()
difficult_list = pd.DataFrame(target_feats, columns=["Target"])
difficult_list["Oof log_loss"] = 0
difficult_list.set_index("Target")

for target in target_feats:
    print(target)
    valid, pred_value, feature_importance_df, mean_score, score = modelling_lgb(train_feat, train_score, test_feat, target)
    train_checkscore[target] = valid
    difficult_list.loc[target, "Oof log_loss"] = score
    print("top10: ", list(feature_importance_df.iloc[:10]["Feature"]))
    print("last10: ", list(feature_importance_df.iloc[-10:]["Feature"]))
    del feature_importance_df
    sub[target] = pred_value

5-alpha_reductase_inhibitor
Fold 1
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	training's binary_logloss: 0.00534643	valid_1's binary_logloss: 0.00529191
Fold 2
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[106]	training's binary_logloss: 0.000741239	valid_1's binary_logloss: 0.00490895
Fold 3
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[79]	training's binary_logloss: 0.00088101	valid_1's binary_logloss: 0.0062801
Fold 4
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[45]	training's binary_logloss: 0.00126784	valid_1's binary_logloss: 0.00655353
Fold 5
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[45]	training's binary_logloss: 0.00143458	valid_1's binary_logloss: 0.00526502
mean log_loss: 0.005659903094615065, oof log_loss= 0.0056

In [7]:
np.mean(difficult_list["Oof log_loss"])

0.007980086071942514

In [8]:
sub.to_csv("submission.csv")